SCRAPING - ss 21 
--------------- 
> Chiara Piccolroaz

> University of Konstanz

> Department of Computer and Information Science; Department of political and administration science


IMPORTING PACKAGES
--------------------------

WEbseiten for Scraping Juris : https://www.juris.de/jportal/portal/ and https://rzblx10.uni-regensburg.de/dbinfo/detail.php?bib_id=ubko&colors=&ocolors=&lett=f&tid=0&titel_id=865

WebSeite for Installing Selenium: : https://www.jcchouinard.com/learn-selenium-python-seo-automation/

In [90]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
import requests, PyPDF2
from io import BytesIO
import pickle 

Go to Juris-website. Make sure to:
1. be locked in the Uni-WLAN or connected with the Uni-VPN
2. the packages have been download

Possible adjustment to-do:
1. remaining logged in. The problem is, that after every x-time, the web-site logged me automatically out --> solve this problem 

In [91]:
driverC=webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe") 
driverC.get('https://www.juris.de/jportal/?action=JLoginUser&username=HEBISUKN.autologin')
driverC.get('https://www.juris.de/r3/search')
driverC.maximize_window()
pickle.dump(driverC.get_cookies() , open("QuoraCookies.pkl","wb"))

for cookie in pickle.load(open("QuoraCookies.pkl", "rb")): 
    driverC.add_cookie(cookie)

Select the relevant filers and buttons. Make sure to:
1. run the sleep functions, otherwise the other functions will override. 
2. the chrome window is maximized
3. the previous function has run 

In [92]:
button = driverC.find_element_by_id("categories_hits_Rechtsprechung")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Sachgebiete")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Asylis___MILo")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_apply")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("linkWorkId80896")
button.click()
time.sleep(2) 

1. Collect the links:

general Procedure: Collect the link of the pdfs for each of the 19151 elements. In order to do that use the following function, which enters each of the 19151 element files, get all PDFs and save them in a list. 

In [57]:
# 0. Initialisation

# the list the PDF-files are gonna be saved in
listPDFs = [] 
# i refers to the number of ducuments pro page and gives the function the id of the document we are looking for. Put i = j = 0
i = 0 
# j refers to the number of documents in total. Put i = j = 0
j = 0
# p refers to the number of the page 
p = 1

# 0. repeat the function for the number of the documents: 
while j < 20491:
    # get the name
    # create the string, which indicates the element_id on the website, which we want to enter the element file with 
    str1 ='resultlistentry_LI' 
    str2 = "{}".format(i)
    str = str1 + str2
    print(str) 
    
# 1. enter the element´s file
    Urteil = driverC.find_element_by_id(str) 
    Urteil.click()
    time.sleep(1) 
    
# 2. get the PDF-link(s) from the respective element´s file
    # get the links and put in the list linkList
    links=driverC.find_elements_by_tag_name("a")
    linkList = [link.get_attribute('href') for link in links]
    time.sleep(1)
    # clean the list linkList, in order to have only strings
    clean_linkList = [i for i in linkList if i is not None]
    # count how many PDFs are, which we are interested in:and according to that:  
    count = 0
    sub = '/jportal/docs/anlage/as/bilder/'
    for link in clean_linkList:
        if sub in link:
            count = count +1
    # if these are more than just one PDF: select the links and add it to the list linkPDF as a list
    linkPDF = []
    if count > 1:
        print('multiple PDFs')
        for link in clean_linkList:
            if sub in link:
                linkPDF.append(link)         
    # if there is only one PDF: select the link and add to the list linkPDF;       
    if count == 1:
        print("got the PDFs, go one back")
        for link in clean_linkList:
            if sub in link:
                linkPDF = link
    # if there are no PDFs: do nothing  
    if count < 1:
        print('no PDF') 
        
# 4. add the PDF(s) in the ListPDFs
    listPDFs.append(linkPDF)
    #print(listPDFs)
    time.sleep(1)
    
# 5. go back to the main page and go on with the other elements
    backtoList = driverC.find_element_by_class_name("tlrahmen")
    backtoList.click()
    time.sleep(3) 
    
# 6. increase i to check for the elements pro page; increase j to check for the 19151 elements
    i = i+1
    j = j+1 
    
# 6. if te last element change the page 
    if i == 25: 
        print("i is equal 25 - we got the 25th element of the page -, so change page")  
        p = p+1
        print("NEXT page" , p)
        driverC.execute_script("window.scrollTo(0, 0);")
        button = driverC.find_element_by_class_name("svg-icon-chevron_right")
        button.click()
        time.sleep(3)  
        i= 0
        

resultlistentry_LI0
got the PDFs, go one back
resultlistentry_LI1
got the PDFs, go one back
resultlistentry_LI2
multiple PDFs
resultlistentry_LI3
got the PDFs, go one back
resultlistentry_LI4
no PDF
resultlistentry_LI5
no PDF
resultlistentry_LI6
got the PDFs, go one back
resultlistentry_LI7
got the PDFs, go one back
resultlistentry_LI8
got the PDFs, go one back
resultlistentry_LI9
got the PDFs, go one back
resultlistentry_LI10
got the PDFs, go one back
resultlistentry_LI11
got the PDFs, go one back
resultlistentry_LI12
no PDF
resultlistentry_LI13
no PDF
resultlistentry_LI14
got the PDFs, go one back
resultlistentry_LI15
got the PDFs, go one back
resultlistentry_LI16
no PDF
resultlistentry_LI17
got the PDFs, go one back
resultlistentry_LI18
got the PDFs, go one back
resultlistentry_LI19
got the PDFs, go one back


WebDriverException: Message: chrome not reachable
  (Session info: chrome=91.0.4472.124)


Have a look at the links, we have selected in the previous function

In [72]:
listPDFs

['https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17486062.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17255995.pdf',
 ['https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/12711966.pdf',
  'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/12711968.pdf'],
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18775353.pdf',
 [],
 [],
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/16730858.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/13989011.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18746441.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18784938.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/19266048.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/13294897.pdf',
 [],
 [],
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17609313.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/12973260.pdf',
 [],
 'https://www.juris.de/

With this function we want to restructure the list of links, by modifing the sublists, that means eleminating the empty  sublistst and ectracting the links from the sublists which connatins multiple informations. 

In [79]:
# eleminating the empty  sublistst
listPDFs_cleaned1 = [x for x in listPDFs if x]

# exctracting the links from the sublists which connatins multiple informations
def flatten(A):
    rt = []
    for i in A:
        if isinstance(i,list): rt.extend(flatten(i))
        else: rt.append(i)
    return rt

listPDFs_cleaned2 = flatten(listPDFs_cleaned1)
listPDFs_cleaned = listPDFs_cleaned2

In [80]:
listPDFs_cleaned

['https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17486062.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17255995.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/12711966.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/12711968.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18775353.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/16730858.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/13989011.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18746441.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18784938.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/19266048.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/13294897.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17609313.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/12973260.pdf',
 'https://www.juris.de/jportal/docs/anlage/as/bilde

Get the PDFs from the links. In order to save space and computational resources, we do not download the PDFs as such, but extract the text of the PDFs and save them into a text file. 

PROBLEM: 
Die Webseite kann nur dann geöffnet werden, wenn man auch in der Webseite ist!!! D.h. diese Funktion in die erste einbauen --> Screaping_9

In [84]:
# probe
import requests, PyPDF2
from io import BytesIO

url = 'https://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17486062.pdf'
response = requests.get(url)
my_raw_data = response.content

with BytesIO(my_raw_data) as data:
    read_pdf = PyPDF2.PdfFileReader(data)

    for page in range(read_pdf.getNumPages()):
        print(read_pdf.getPage(page).extractText())

PdfReadError: Cannot read an empty file

In [82]:
# funktioniert nicht
TextList = []

for link in listPDFs_cleaned:
    Pages = []
    response = requests.get(link)
    my_raw_data = response.content
    
    with BytesIO(my_raw_data) as data:
        read_pdf = PyPDF2.PdfFileReader(data)
        
        for page in range(read_pdf.getNumPages()):
            p =read_pdf.getPage(page).extractText()
            p = p.replace("\n", "")
            Pages.append(p)
        JointPages = "".join(Pages)
        TextList.append(JointPages)

PdfReadError: Cannot read an empty file

In [10]:
TextList

'  PDF Test File  Congratulations, your computer is equipped with a PDF (Portable Document Format) reader!  You should be able to view any of the PDF documents and forms available on our site.  PDF forms are indicated by these icons: \n  or  \n.    Yukon Department of Education Box 2703 Whitehorse,Yukon Canada Y1A 2C6  Please visit our website at:  \nhttp://www.education.gov.yk.ca/\n   '

In [2]:
with open('test.txt', 'w') as f:
    for item in TextList:
        f.write("%s\r\n" % item)

NameError: name 'TextList' is not defined